# Aula prática: Bin Packing


## Exercício 1


### Descrição do problema
Uma faculdade quer agendar as provas de 6 disciplinas. Existe uma regra que proíbe que o mesmo aluno tenha que fazer mais de uma prova por dia. A tabela abaixo mostra quais alunos (identificados por números) vão fazer a prova de cada disciplina. Por exemplo, A e B não podem ser agendadas no mesmo dia por causa do aluno 1. Já B e C poderiam ser agendadas no mesmo dia, pois nenhum aluno vai fazer essas duas provas.

| Disciplina | Alunos |
|:---:|:---:|
| A | 1, 2, 3, 4 |
| B | 1, 5 |
| C | 3, 6, 7 |
| D | 5, 7 |
| E | 2, 7 |
| F | 4, 5 |

Crie um modelo de PLI para agendar essas provas no menor número de dias possível.
**Dica: baseie-se no modelo do problema de empacotamento de caixas.**


### Resolução

In [8]:
# instalação e importação do pacote mip

from mip import *

# funcões usadas posteriormente:

# resolve o modelo e mostra os valores das variáveis
def solve(model):
  status = model.optimize()

  print("Status = ", status)
  if status != OptimizationStatus.OPTIMAL:
    return

  print(f"Solution value  = {model.objective_value:.2f}\n")

  print("Solution:")
  for v in model.vars:
    if v.x > 0.001:
      print(f"{v.name} = {v.x:.2f}")


# salva modelo em arquivo lp, e mostra o conteúdo
def save(model, filename):
  model.write(filename) # salva modelo em arquivo
  with open(filename, "r") as f: # lê e exibe conteúdo do arquivo
    print(f.read())

In [9]:
dic = {'A':[1,2,3,4],'B':[1,5],'C':[3,6,7],'D':[5,7],'E':[2,7],'F':[4,5]}

I = [(1,2),(2,4,6),(1,5),(1,6),(1,3),(3,4,5)]

for k in I:
  print(k,len(k))

(1, 2) 2
(2, 4, 6) 3
(1, 5) 2
(1, 6) 2
(1, 3) 2
(3, 4, 5) 3


In [10]:
#x[1][1] + x[2][1]
#x[1][2] + x[2][2]
# ...  +   ...
#x[1][6] + x[2][6]




In [11]:
model = Model(sense=MINIMIZE, solver_name=CBC)

y = {i: model.add_var(var_type=INTEGER, name=f'y_{i}', lb=0.0) for i in range(1,7)}
x = {i: {j: model.add_var(var_type=INTEGER, name=f'x_{i}_{j}', lb=0.0) for j in range(1,7)}  for i in range(1,7)}

model.objective = xsum(y[i]for i in range(1,7))

p = [1,2,3,4,5,6]
d = [1,2,3,4,5,6]

# Restrição 1 - Somatório de (j pertence a d) (xij) = 1, para todo i pertence a p

for i in range(1,7):
  model += xsum(x[i][j] for j in range(1,7)) == 1

""""
for j in range(1,7):
  model += x[1][j] + x[2][j] <= y[j]
  model += x[1][j] + x[3][j] <= y[j]
  model += x[1][j] + x[5][j] <= y[j]
  model += x[1][j] + x[6][j] <= y[j]
  
  model += x[3][j] + x[4][j] + x[5][j] <= y[j]
  model += x[2][j] + x[4][j] + x[6][j] <= y[j]
  
"""


for j in range(1,7):
  for k in I:
    model += xsum(x[i][j] for i in k) <= y[j]
    
for i in range(1,7):
  model+= y[i] <= 1
  for j in range(1,7):
    model += x[i][j] <= y[j]

save(model, "model.lp")

\Problem name: 

Minimize
OBJROW: y_1 + y_2 + y_3 + y_4 + y_5 + y_6
Subject To
constr(0):  x_1_1 + x_1_2 + x_1_3 + x_1_4 + x_1_5 + x_1_6 = 1
constr(1):  x_2_1 + x_2_2 + x_2_3 + x_2_4 + x_2_5 + x_2_6 = 1
constr(2):  x_3_1 + x_3_2 + x_3_3 + x_3_4 + x_3_5 + x_3_6 = 1
constr(3):  x_4_1 + x_4_2 + x_4_3 + x_4_4 + x_4_5 + x_4_6 = 1
constr(4):  x_5_1 + x_5_2 + x_5_3 + x_5_4 + x_5_5 + x_5_6 = 1
constr(5):  x_6_1 + x_6_2 + x_6_3 + x_6_4 + x_6_5 + x_6_6 = 1
constr(6):  - y_1 + x_1_1 + x_2_1 <= -0
constr(7):  - y_1 + x_2_1 + x_4_1 + x_6_1 <= -0
constr(8):  - y_1 + x_1_1 + x_5_1 <= -0
constr(9):  - y_1 + x_1_1 + x_6_1 <= -0
constr(10):  - y_1 + x_1_1 + x_3_1 <= -0
constr(11):  - y_1 + x_3_1 + x_4_1 + x_5_1 <= -0
constr(12):  - y_2 + x_1_2 + x_2_2 <= -0
constr(13):  - y_2 + x_2_2 + x_4_2 + x_6_2 <= -0
constr(14):  - y_2 + x_1_2 + x_5_2 <= -0
constr(15):  - y_2 + x_1_2 + x_6_2 <= -0
constr(16):  - y_2 + x_1_2 + x_3_2 <= -0
constr(17):  - y_2 + x_3_2 + x_4_2 + x_5_2 <= -0
constr(18):  - y_3 + x_1_3 + 

In [12]:
solve(model)

Status =  OptimizationStatus.OPTIMAL
Solution value  = 3.00

Solution:
y_2 = 1.00
y_4 = 1.00
y_6 = 1.00
x_1_4 = 1.00
x_2_6 = 1.00
x_3_6 = 1.00
x_4_4 = 1.00
x_5_2 = 1.00
x_6_2 = 1.00
